Reading the json file and data i.e., iris.csv file

In [1]:
# importing necessary libraries
import pandas as pd
import json

In [ ]:
#reading the given json file into the notebook.
with open('algoparams_from_ui.json', 'r') as json_file:
    data = json.load(json_file)
data

In [3]:
# loading the dataset into the notebook.
iris = pd.read_csv('/content/iris.csv')

1) Read the target and type of regression to be run

In [4]:
# slecting the target feature as mentioned in the json file.
target = data['design_state_data']['target']['target']

In [5]:
# reading the type of operation
ty = data['design_state_data']['target']['type']
ty

'regression'

In [6]:
# Since the type is reggression we can ignore the species feature. It is not a continous feature.(it is a discrete categorical feature)
if ty == 'regression':
  iris = iris.drop('species', axis=1)

In [7]:
# splitting the data as X - independent features and y - dependent feature
X = iris.drop(target,axis=1 ).values
y = iris[target].values

In [ ]:
# printing X and y to check they where splitted correctly
X, y

2) Read the features (which are column names in the csv) and figure out what missing imputation needs to be applied and apply that to the columns loaded in a dataframe

In [9]:
# When the given type is classification we need to encode the species feature otherwise not needed.
if ty == 'classification':
  from sklearn.preprocessing import LabelEncoder
  le = LabelEncoder()
  iris['species'] = le.fit_transform(iris[['species']])

In [10]:
# Performing Imutation of missing values and feature scaling.
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np
# Feature Handling
preprocessed_features = {}
for feature_name, feature_data in data['design_state_data']['feature_handling'].items():
    feature_details = feature_data['feature_details']
    if 'missing_values' in feature_details:
        missing_values = feature_details['missing_values']
        impute_method = missing_values if isinstance(missing_values, str) else missing_values['impute_with']
        if impute_method == 'Impute':
            if isinstance(missing_values, str):
                impute_value = 0
            else:
                impute_value = missing_values['impute_value']
            imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean', fill_value = impute_value)
            for column in iris.columns:
              iris[column] = imputer.fit_transform(iris[[column]])
    else:
        impute_method = None
    if 'rescaling' in feature_details:
        rescaling_method = feature_details['rescaling']
        if rescaling_method == 'Standardization':
            scaler = StandardScaler()
            for column in target.columns:
              target[column] = scaler.fit_transform(target[[column]])
    else:
        rescaling_method = None

3) Compute feature reduction based on input. See the screenshot below where there can be No Reduction, Corr with Target, Tree-based, PCA. Please make sure you write code so that all options can work. If we rerun your code with a different Json it should work if we switch No Reduction to say PCA.


In [11]:
#pca
import json
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Extract options from input
reduction_option = data['design_state_data']['feature_reduction']['feature_reduction_method']
n_features_to_keep = data['design_state_data']['feature_reduction']['num_of_features_to_keep']
n_components = 2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Defining a function to perform the specified reduction technique
def perform_feature_reduction(X_train, X_test, y_train, reduction_option, n_features_to_keep=None, n_components=None):
    if reduction_option == "No Reduction":
        return X_train, X_test
    elif reduction_option == "Corr with Target":
        # Selecting top features based on correlation with target
        selector = SelectKBest(score_func=f_classif, k=n_features_to_keep)
        selector.fit(X_train, y_train)
        X_train = selector.transform(X_train)
        X_test = selector.transform(X_test)
        return X_train, X_test
    elif reduction_option == "Tree-based":
        # Selecting top features based on feature importances from a tree-based model
        model = RandomForestRegressor(random_state=42)
        model.fit(X_train, y_train)
        feature_importances = model.feature_importances_
        top_features_indices = np.argsort(feature_importances)[::-1][:int(n_features_to_keep)]
        print("X_train shape before slicing:", X_train.shape)
        print("X_test shape before slicing:", X_test.shape)
        print("top_features_indices:", top_features_indices)
        X_train = X_train[:, top_features_indices]
        X_test = X_test[:, top_features_indices]
        print("X_train shape after slicing:", X_train.shape)
        print("X_test shape after slicing:", X_test.shape)
        return X_train, X_test
    elif reduction_option == "PCA":
        # Performing PCA to reduce dimensions
        pca = PCA(n_components=n_components)
        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)
        return X_train, X_test
# Performing the specified feature reduction
X_train_reduced, X_test_reduced = perform_feature_reduction(X_train, X_test, y_train, reduction_option, n_features_to_keep, n_components)
# Now we can use X_train_reduced and X_test_reduced for further modeling and analysis


X_train shape before slicing: (120, 3)
X_test shape before slicing: (30, 3)
top_features_indices: [2 1 0]
X_train shape after slicing: (120, 3)
X_test shape after slicing: (30, 3)


4) Parse the Json and make the model objects (using sklean) that can handle what is required in the “prediction_type” specified in the JSON (See #1 where “prediction_type” is specified). Keep in mind not to pick models that don’t apply for the prediction_type specified


5) Run the fit and predict on each model – keep in mind that you need to do hyper parameter tuning i.e., use GridSearchCV

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor,ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet
from xgboost import XGBClassifier, XGBRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

# Iterating through the algorithms and their parameters
for algo_name, algo_params in data['design_state_data']['algorithms'].items():
    if algo_params['is_selected']:
        print(f"Training {algo_name}...")
        if algo_name == "RandomForestClassifier":
          model = RandomForestClassifier()
          param_grid = {
        'n_estimators': range(algo_params['min_trees'], algo_params['max_trees'] + 1),
        'max_depth': range(algo_params['min_depth'], algo_params['max_depth'] + 1),
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': range(algo_params['min_samples_per_leaf_min_value'], algo_params['min_samples_per_leaf_max_value'] + 1),
        #'max_features': ['auto', 1.0],

    }
        elif algo_name == "RandomForestRegressor":
            model = RandomForestRegressor()
            param_grid = {
        'n_estimators': range(algo_params['min_trees'], algo_params['max_trees'] + 1),
        'max_depth': range(algo_params['min_depth'], algo_params['max_depth'] + 1),
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': range(algo_params['min_samples_per_leaf_min_value'], algo_params['min_samples_per_leaf_max_value'] + 1),
        #'max_features': ['auto', 1.0],

    }
        elif algo_name == "GBTClassifier":
            model = GradientBoostingClassifier()
            param_grid = {
        'n_estimators': algo_params['num_of_BoostingStages'],
        'learning_rate': algo_params['learningRate'],
        'subsample': [0.8, 1.0],
        'max_depth': range(algo_params['min_depth'], algo_params['max_depth'] + 1),
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],

    }
        elif algo_name == "GBTRegressor":
            model = GradientBoostingRegressor()
            param_grid = {
        'n_estimators': algo_params['num_of_BoostingStages'],
        'subsample': [0.8, 1.0],
        'max_depth': range(algo_params['min_depth'], algo_params['max_depth'] + 1),
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],

    }
        elif algo_name == "LinearRegression":
            model = LinearRegression()
            param_grid = {
        'fit_intercept': [True, False],
        'normalize': [True, False],

    }
        elif algo_name == "LogisticRegression":
          model = LogisticRegression()
          param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 0.1, 1, 10],
        'fit_intercept': [True, False],
        'solver': ['liblinear'],

    }
        elif algo_name == "RidgeRegression":
            model = Ridge()
            param_grid = {
        'alpha': np.linspace(algo_params['min_regparam'], algo_params['max_regparam'], num=5),
        'fit_intercept': [True, False],
        'normalize': [True, False],
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],

    }
        elif algo_name == "LassoRegression":
            model = Lasso()
            param_grid = {
        'alpha': np.linspace(algo_params['min_regparam'], algo_params['max_regparam'], num=5),
        'fit_intercept': [True, False],
        'normalize': [True, False],

    }
        elif algo_name == "ElasticNetRegression":
            model = ElasticNet()
            param_grid = {
        'alpha': np.linspace(algo_params['min_regparam'], algo_params['max_regparam'], num=5),
        'l1_ratio': np.linspace(algo_params['min_elasticnet'], algo_params['max_elasticnet'], num=5),
        'fit_intercept': [True, False],
        'normalize': [True, False],

    }
        elif algo_name == "xg_boost":
            model = XGBClassifier() if algo_params['use_gradient_boosted_tree'] else XGBRegressor()
            param_grid = {
        'n_estimators': algo_params['max_num_of_trees'],
        'learning_rate': algo_params['learningRate'],
        'max_depth': algo_params['max_depth_of_tree'],
        'gamma': algo_params['gamma'],
        'min_child_weight': algo_params['min_child_weight'],
        'subsample': algo_params['sub_sample'],
        'colsample_bytree': algo_params['col_sample_by_tree'],

    }
        elif algo_name == "DecisionTreeRegressor":
            model = DecisionTreeRegressor()
            param_grid = {
        'max_depth': range(algo_params['min_depth'], algo_params['max_depth'] + 1),
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': algo_params['min_samples_per_leaf'],
        'criterion': ['mse', 'friedman_mse', 'mae'],

    }
        elif algo_name == "DecisionTreeClassifier":
            model = DecisionTreeClassifier()
            param_grid = {
        'max_depth': range(algo_params['min_depth'], algo_params['max_depth'] + 1),
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': algo_params['min_samples_per_leaf'],
        'criterion': ['gini', 'entropy'],

    }
        elif algo_name == "SVM":
            model = SVC() if algo_params['linear_kernel'] else SVR()
            param_grid = {
        'C': algo_params['c_value'],
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'degree': [2, 3, 4],
        'coef0': [0.0, 1.0, 2.0],
        'gamma': ['scale', 'auto'] if algo_params['auto'] else algo_params['custom_gamma_values'],

    }
        elif algo_name == "SGD":
            model = SGDClassifier() if algo_params['use_logistics'] else SGDRegressor()
            param_grid = {
        'loss': ['hinge', 'log', 'modified_huber'],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'alpha': algo_params['alpha_value'],
        'l1_ratio': [0.15, 0.25, 0.35, 0.45, 0.55],
        'fit_intercept': [True, False],
        'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],

    }
        elif algo_name == "KNN":
            model = KNeighborsClassifier() if algo_params['k_value'] else KNeighborsRegressor()
            param_grid = {
        'n_neighbors': algo_params['k_value'],
        'weights': ['uniform', 'distance'] if algo_params['distance_weighting'] else ['uniform'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'p': [1, 2, 3],

    }
        elif algo_name == "extra_random_trees":
            model = ExtraTreesClassifier() if algo_params['is_selected'] else ExtraTreesRegressor()
            param_grid = {
        'n_estimators': algo_params['num_of_trees'],
        'max_depth': algo_params['max_depth'],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': algo_params['min_samples_per_leaf'],
        #'max_features': ['auto', 1.0],

    }
        elif algo_name == "neural_network":
            model = MLPClassifier() if algo_params['is_selected'] else MLPRegressor()
            param_grid = {
        'hidden_layer_sizes': algo_params['hidden_layer_sizes'],
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
        'learning_rate': ['constant', 'invscaling', 'adaptive'],
        'solver': ['lbfgs', 'sgd', 'adam'],
        'early_stopping': [True, False],

    }

        # Creating GridSearchCV for parameter tuning
        grid_search = GridSearchCV(model, param_grid, cv=5)
        grid_search.fit(X_train_reduced, y_train)

        # Get the best model and evaluate on the test set
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test_reduced)

        # Evaluate the model based on the problem type (classification or regression)
        if algo_name in ["RandomForestClassifier", "GBTClassifier", "DecisionTreeClassifier", "LogisticRegression", "SVM", "SGD", "KNN", "extra_random_trees"]:
            accuracy = accuracy_score(y_test, y_pred)
            print(f"{algo_name} - Accuracy: {accuracy:.2f}")
        elif algo_name in ["RandomForestRegressor", "GBTRegressor", "DecisionTreeRegressor", "LinearRegression", "RidgeRegression", "LassoRegression", "ElasticNetRegression", "xg_boost", "neural_network"]:
            mse = mean_squared_error(y_test, y_pred)
            print(f"{algo_name} - Mean Squared Error: {mse:.2f}")
            print(f"{algo_name} - r2 score: {r2_score(y_test, y_pred)}")




Training RandomForestRegressor...
RandomForestRegressor - Mean Squared Error: 0.04
RandomForestRegressor - r2 score: 0.9423272002624384
